# **Procedimiento para realizar consultas (request) a la base de datos de la API de EDEMAM**
## **Estudiante: Samuel Cortez / Maestría en Ciencia de Datos**

In [90]:
import pandas as pd
import requests
import json         
import logging

In [88]:
# Se define clase Edemam_mcd
class Edamam_mcd:
    "API que regresa datos de tipo json"
    def __init__(self, 
                id_nutrition, 
                key_nutrition, 
                id_recipes,
                key_recipes,
                id_food,
                key_food):
        
        # Asignación de claves / ID de las API's
        self.id_nut = id_nutrition
        self.key_nut = key_nutrition
        self.id_rec = id_recipes
        self.key_rec = key_recipes
        self.id_food = id_food
        self.key_food = key_food
    
    def Nut_Analysis(self, query):
        self.Nut_ingredients = query
        # Se define url para realizar consulta a la API Nutrition Analysis
        url_req_nutr = 'https://api.edamam.com/api/nutrition-data?app_id={id}&app_key={key}'.format(id=self.id_nut,key= self.key_nut)+'&nutrition-type=cooking&ingr='+self.Nut_ingredients
        # Envío de solicitud para realizar consulta de análisis nutricional
        Response = requests.get(url_req_nutr)

        # Imprimir el código del estatus
        print("Status Code (Nutrition Analysis API):", Response.status_code)
        
        if Response.status_code == 401:
            logging.error('{key} - Clave inválida (Nutrition Analysis API)'.format(key=self.key_nut))
        # Convertir como Json
        self.r_Nut = Response.json()
        return self.r_Nut

    def Search_recipe(self, query):
        self.query_recipe = query
        # Se define url para realizar consulta para busqueda de recetas de un platillo
        url_recipe = 'https://api.edamam.com/search?q={query}&app_id={id}&app_key={key}'.format(id=self.id_rec, key=self.key_rec,query=self.query_recipe)    
        Response = requests.get(url_recipe)

        # Imprimir el código de estatus
        print("Status Code (Recipe Search API) :", Response.status_code)
        
        if Response.status_code == 401:
            logging.error('{key} - Clave inválida (Recipe Search API)'.format(key=self.key_rec))
        # Convertir a Json
        self.r_Recipe = Response.json()

        return self.r_Recipe

    def Search_food(self, query):
        self.query_food = query
        # Se define url para realizar consulta de análisis de comida
        url_food = 'https://api.edamam.com/api/food-database/parser?nutrition-type=logging&ingr={query}&app_id={id}&app_key={key}' \
            .format(id=self.id_food, key=self.key_food, query=self.query_food)
        # Se realiza una consulta dando la url
        Response = requests.get(url_food)
        
        # Imprimir el código de estatus
        print("Status Code (Food Database API):", Response.status_code)

        if Response.status_code == 401:
            logging.error('{key} - Clave inválida (Food Database API)'.format(key=self.key_rec))

        # Convertir a json
        self.r_Food = Response.json()
        return self.r_Food
    
    # Conjunto de funciones para generar dataframes con los datos principales de las consultas

    def Nutrient_Guide(self):
        # Manejo de excepciones
        try:
            # Se crean DataFrames de las variables totalNutrients, totalDaily y totalNutrientsKCal 
            self.df_Nutrition= pd.DataFrame(self.r_Nut.get('totalNutrients')).T.rename_axis(str(self.Nut_ingredients))
            self.df_totalDaily = pd.DataFrame(self.r_Nut.get('totalDaily')).T.rename_axis(str(self.Nut_ingredients))
            self.df_total_Nut =  pd.DataFrame(Response_Nut.get('totalNutrientsKCal')).T.rename_axis(str(self.Nut_ingredients))
            # Se obtiene las calorías del alimento
            self.Nutrient_Cal = self.r_Nut.get('calories')
            # Se obtiene el peso total del alimento
            self.totalWeight = self.r_Nut.get('totalWeight')
        except Exception as e:
            print(e)
        
        
    def ingredients_table(self):
        # Manejo de excepciones
        try:
            # Comprensión de listas: se crean nuevas listas con las etiquetas de la receta y con los ingredientes
            list_label = [element_hits.get('recipe').get('label') for element_hits in Response_Recipe.get('hits')]
            list_ingredientes = [element_ingre.get('recipe').get('ingredients') for element_ingre in Response_Recipe.get('hits')]
            # Se construye DataFrame 
            df_ = pd.DataFrame(list_ingredientes, index=list_label).stack().apply(pd.Series).drop(columns=['foodId'])
            self.df_Recipe = df_.rename_axis(["Types", "Items"], axis = "rows").rename(columns = {"foodCategory": "food category", "text":"ingredient name"})
            
            return self.df_Recipe
        except Exception as e:
            print(e)

    def food_table(self):
        # Se generar listas vacias
        list_foods = []
        list_nutrients = []
        rename_nutrients = ['ENERGY (kcal)', 'PROTEIN (g)', 'FAT (g)', 'CARBS (g)', 'FIBER (g)']
        # Manejo de excepciones
        try:
            # Comprensión de listas: se crean nuevas listas con las etiquetas de los alimentos y con los nutrientes  
            list_foods = [element.get('food').get('label') for  element in self.r_Food['hints']]
            list_nutrients = [element.get('food').get('nutrients') for element in self.r_Food['hints']]
            # Se construye DataFrame
            self.df_food_table = pd.DataFrame(list_nutrients, index = list_foods).round(2).fillna('Unknown')
            self.df_food_table.columns = rename_nutrients
            return self.df_food_table
            
        except Exception as e:
            print(e) 
    
    def write_files(self):
        #  Exportar archivos .CSV/.xlsx
        path_Nutrition = '{Nut}_Nutritional_Analysis.xlsx'.format(Nut=self.Nut_ingredients)
        path_Recipe = '{Recipe}_Recipe.csv'.format(Recipe = self.query_recipe)
        path_food = '{food}_food.csv'.format(food = self.query_food)

        # Se exporta a excel el análisis nutricional 
        with pd.ExcelWriter(path_Nutrition) as writer:
            self.df_Nutrition.to_excel(writer, sheet_name= 'Nutritional_Analysis')
            self.df_totalDaily.to_excel(writer, sheet_name='Total_Daily')
            self.df_total_Nut.to_excel(writer, sheet_name='totalNutrientsKCal')
        # Se exporta en formato CSV los ingredientes de las recetas
        self.df_Recipe.to_csv(path_Recipe)
        # Se exporta en CSV el dataframe con el análisis de los alimentos
        self.df_food_table.to_csv(path_food)



In [92]:
# Claves / ID's de las aplicaciones asignadas
nutrition_appid= '5f1e7888'
nutrition_appkey= '5576413f5acb38f5259a58dad888d1b9'
recipes_appid = 'dcda2d5e'
recipes_appkey= '87ae2d34cdde47026fc341c8fbd7957f'
food_appid= '6b69c52c'
food_appkey= 'd5a448b534a0c358dfb47fa7a6548724'

# Se crea el objeto 
EDEMAM_consulta = Edamam_mcd(id_nutrition=nutrition_appid, 
                            key_nutrition= nutrition_appkey,
                            id_recipes=recipes_appid,
                            key_recipes=recipes_appkey,
                            id_food=food_appid,
                            key_food=food_appkey)

# Ejercución de los métodos de cada una de las API's para realizar consultas
Response_Nut = EDEMAM_consulta.Nut_Analysis(query= '2 hams')
Response_Food = EDEMAM_consulta.Search_food(query= 'coffe and pizza')
Response_Recipe = EDEMAM_consulta.Search_recipe(query= 'Twinkies')
# Ejecución de los métodos para obtener los dataframe con los resultados para cada una de las API's
EDEMAM_consulta.Nutrient_Guide()
EDEMAM_consulta.ingredients_table()
EDEMAM_consulta.food_table()
# Se exportan en archivos csv y xlsx los resultados obtenidos
EDEMAM_consulta.write_files()


Status Code (Nutrition Analysis API): 200
Status Code (Food Database API): 200
Status Code (Recipe Search API) : 200


In [93]:
# Consulta a la API: Nutrition Analysis obtenida en formato json
Response_Nut

{'uri': 'http://www.edamam.com/ontologies/edamam.owl#recipe_95620f46a47a4a7996864aecec9ac91c',
 'calories': 29354,
 'totalWeight': 16308.0,
 'dietLabels': ['LOW_CARB'],
 'healthLabels': ['SUGAR_CONSCIOUS',
  'LOW_SUGAR',
  'PALEO',
  'DAIRY_FREE',
  'GLUTEN_FREE',
  'WHEAT_FREE',
  'EGG_FREE',
  'MILK_FREE',
  'PEANUT_FREE',
  'TREE_NUT_FREE',
  'SOY_FREE',
  'FISH_FREE',
  'SHELLFISH_FREE',
  'CRUSTACEAN_FREE',
  'CELERY_FREE',
  'MUSTARD_FREE',
  'SESAME_FREE',
  'LUPINE_FREE',
  'MOLLUSK_FREE',
  'ALCOHOL_FREE',
  'NO_OIL_ADDED',
  'NO_SUGAR_ADDED',
  'SULPHITE_FREE'],
 'cautions': ['SULFITES'],
 'totalNutrients': {'ENERC_KCAL': {'label': 'Energy',
   'quantity': 29354.4,
   'unit': 'kcal'},
  'FAT': {'label': 'Fat', 'quantity': 1679.7240000000002, 'unit': 'g'},
  'FASAT': {'label': 'Saturated', 'quantity': 558.0597600000001, 'unit': 'g'},
  'FAMS': {'label': 'Monounsaturated',
   'quantity': 798.6027600000001,
   'unit': 'g'},
  'FAPU': {'label': 'Polyunsaturated',
   'quantity': 2

In [96]:
# Consulta a la API: Food Database obtenida en formato json
Response_Food

{'text': 'coffe and pizza',
 'parsed': [{'food': {'foodId': 'food_at830s9amds32fb8w6ufmaopzk8n',
    'label': 'Pizza',
    'nutrients': {'ENERC_KCAL': 268.0,
     'PROCNT': 10.36,
     'FAT': 12.28,
     'CHOCDF': 29.02,
     'FIBTG': 2.2},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/c01/c0150280d71059c23c025f501f502dc0.jpg'},
   'quantity': 1.0,
   'measure': {'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_serving',
    'label': 'Serving'}}],
 'hints': [{'food': {'foodId': 'food_at830s9amds32fb8w6ufmaopzk8n',
    'label': 'Pizza',
    'nutrients': {'ENERC_KCAL': 268.0,
     'PROCNT': 10.36,
     'FAT': 12.28,
     'CHOCDF': 29.02,
     'FIBTG': 2.2},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/c01/c0150280d71059c23c025f501f502dc0.jpg'},
   'measures': [{'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_unit',
     'label': 'Whole'},
   

In [ ]:
Response_Recipe